In [655]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *
from _rulesbuilding import *

In [656]:
print(config.inputs['data'])

{'filename': '01_raw.csv', 'filedir': '/home/jupyter/data', 'fullpath': '/home/jupyter/data/01_raw.csv'}


In [657]:
rules = pd.read_csv(config.inputs['rules']['fullpath'])
rules = rules[(rules['text_match'].notna()) & (rules['text_exclude'].notna() == False)]

regexer = GroupedDataRegex('|')

matchers = []
rdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'M', '(', ')'))
sdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'S', '^(', ').*$'))
adecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'A', '^.*(', ').*$'))

regexes = {}
for matcher in matchers:
    type = rules['matching']
    regex = rules[rules['matching'] == matcher.getType()][['service_id','text_match']]
    regex = matcher.make(regex.groupby('service_id')['text_match']).to_dict()
    inv_map = {v: k for k, v in regex.items()}
    regexes = {**regexes, **inv_map}

In [658]:
signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.inputs['signals'].items()}

In [659]:
data = pd.read_csv(config.inputs['data']['fullpath'])

In [660]:
data = data[data['status'] != 'D']

In [661]:
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [662]:
data['service_id'] = data[['description']]
data['service_id'] = data[['service_id']].replace({'service_id':regexes}, regex=True)

In [663]:
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

In [664]:
csv_data = data.to_csv()

In [665]:
save_file(config.inputs['output']['local']['fullpath'], csv_data)

In [666]:
from google.cloud import storage

client = storage.Client()

output_config = config.inputs['output']
bucket = client.get_bucket(output_config['cloud']['bucket_name'])

blob = bucket.blob(output_config['local']['filename'])

source_file_name = output_config['local']['filename']
blob.upload_from_filename(output_config['local']['fullpath'])

output(blob.public_url)

'https://storage.googleapis.com/antenna-task/processed_data.csv'


In [667]:
# print(data.to_string())